# Analys av könsfördelningen bland vd:ar i svenska aktiebolag

av [Anna Norberg](https://twitter.com/annanorberg)

Hur ser könsfördelningen ut bland vd:ar för aktiebolag i landets kommuner? Nyhetsbyrån Siren har gjort en analys av Bolagsverkets öppna data om företrädare i svenska företag, som finns dokumenterad nedan.

I beräkningen ingår alla aktiebolag i Sverige i april 2018 där en vd finns utsedd. Endast publika aktiebolag måste utse en vd. Privata aktiebolag kan utse en vd, men behöver inte göra det.

Resultatet från analysen har rapporterats i en artikel som skickades ut till Nyhetsbyrån Sirens kunder den 12 april 2018.

## Så här gjorde vi

### Importera Pythons analysverktyg

In [1]:
import pandas as pd

### Läs in data från Bolagsverkets hemsida

In [2]:
heads = pd.read_csv("http://www.bolagsverket.se/statistik/foretradare_historik.csv", engine='python', delimiter='\;')

Visa de fem första raderna i datasetet

In [3]:
heads.head()

,Ar,Foretagsform,Lan,Kommun,PrivatPublikt,Arbetstagarrepresentant,Utlandsbosatt,Bosatt_EES,Kon,Alder_intervall,...,Antal_REVS,Antal_REVSL,Antal_REVST,Antal_REVT,Antal_SU,Antal_SVD,Antal_VD,Antal_VLE,Antal_VOF,Antal_VVD
0,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,50 �r och �ldre,...,10,0,0,0,221,0,6,0,0,2
1,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Kvinna,Yngre �n 50 �r,...,18,1,0,0,180,0,8,0,0,0
2,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,50 �r och �ldre,...,51,11,0,0,197,0,100,0,0,3
3,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Man,Yngre �n 50 �r,...,214,0,0,0,128,0,87,0,0,5
4,2003,AB,Blekinge l�n,Karlshamn kommun,PRIV,,,,Org,Ok�nd,...,0,0,0,0,0,0,0,0,0,0


### Förbered datan för analys

Filtrera ut endast aktiebolag.

In [4]:
aktiebolag = heads[heads.Foretagsform == "AB"]

Filtrera ut aktuellt år.

In [5]:
ab_arton = aktiebolag[aktiebolag.Ar == 2018]

Filtrera ut företag med fysiska företrädare.

In [6]:
ab_arton_fys = ab_arton[ab_arton.JurFys == "F"]

Summera antalet vd per kön och kommun.

In [7]:
vd= ab_arton_fys.groupby(["Kommun", "Kon"]).Antal_VD.sum().reset_index()

In [8]:
kommuner = vd.pivot(index="Kommun", columns="Kon")

In [9]:
pd.set_option('precision', 0)

Nu ser datasetet ut så här:

In [10]:
kommuner.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, Ale kommun to �vertorne� kommun
Data columns (total 3 columns):
(Antal_VD, Kvinna)    290 non-null float64
(Antal_VD, Man)       290 non-null float64
(Antal_VD, Ok�nt)     288 non-null float64
dtypes: float64(3)
memory usage: 9.1+ KB


In [11]:
kommuner.head()

Antal_VD           
Kon               Kvinna  Man Ok�nt
Kommun                             
Ale kommun            28  152     1
Alings�s kommun       73  377     2
Alvesta kommun         9  112     3
Aneby kommun           4   35     3
Arboga kommun         16   89     0

Skippa kolumnen "okänt".

In [12]:
kommuner.drop(kommuner.columns[[2]], axis=1, inplace=True)

### Beräkna totalt antal vd:ar, samt andelen kvinnor och män

In [13]:
kommuner.loc['Summa']= kommuner.sum()

In [14]:
kommuner['Totalt'] = kommuner.sum(axis=1)

In [15]:
kommuner.columns = ['Kvinna', 'Man', 'Totalt']

In [16]:
kommuner["Andel kvinnor"] = (kommuner.Kvinna / kommuner.Totalt)*100

In [17]:
kommuner["Andel män"] = (kommuner.Man / kommuner.Totalt)*100

Nu ser datasetet ut så här:

In [18]:
kommuner.head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
Ale kommun,28,152,180,16,84
Alings�s kommun,73,377,450,16,84
Alvesta kommun,9,112,121,7,93
Aneby kommun,4,35,39,10,90
Arboga kommun,16,89,105,15,85


### Rangordna kommunerna

Här är andelen kvinnor lägst:

In [19]:
kommuner.sort_values("Andel kvinnor").head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
Munkfors kommun,0,8,8,0,100
H�gsby kommun,1,37,38,3,97
Hagfors kommun,2,64,66,3,97
Hylte kommun,3,58,61,5,95
Hjo kommun,3,55,58,5,95


Här är andelen kvinnor högst:

In [20]:
kommuner.sort_values("Andel kvinnor", ascending=False).head()

,Kvinna,Man,Totalt,Andel kvinnor,Andel män
Kommun,,,,,
�sele kommun,11,19,30,37,63
Danderyd kommun,393,854,1247,32,68
Eker� kommun,136,343,479,28,72
H�gan�s kommun,108,274,382,28,72
Liding� kommun,323,825,1148,28,72


### Exportera det nya datasetet till csv

In [21]:
kommuner.to_csv("vd.csv", decimal=',', sep=',', float_format='%.0f')